## Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install dependencies

In [ ]:
!pip install ijson unzip --quiet

In [ ]:
!pip install -U transformers
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes

!pip install triton
!pip install tiktoken

!pip install sentence-transformers
!pip install flash-attn --no-build-isolation

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 102.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

## Import libraries

In [ ]:
import torch
import zipfile
import pandas as pd

from tqdm import tqdm
from pathlib import Path
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification

## Unzip dataset

In [ ]:
zip_path = "/content/drive/MyDrive/IMD - Italian Medical Data/IMD-QA.zip"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("IMD-QA")

In [ ]:
original_df = pd.read_csv("IMD-QA/QA_prima_della_rifo.csv")
reformulated_df = pd.read_csv("IMD-QA/QA_rif_cleaned.csv")

In [ ]:
original_df.head(5)

,ID,URL,data_domanda,data_risposta,autore_domanda,autore_risposta,testo_domanda,testo_risposta,categoria,sito,categoria_generale
0,1,https://www.medicitalia.it/consulti/psicoterap...,2007-12-16,2007-12-16,Autore sconosciuto,Dr. Francesco Saverio Ruggiero Referente Scien...,Psichiatri: per fare psicoterapia necessitano ...,"Gentile utente, uno psichiatra dopo la special...",Psicoterapia,MediciItalia,Salute Mentale
1,2,https://www.medicitalia.it/consulti/psicoterap...,2007-02-08,2007-02-08,Autore sconosciuto,Autore sconosciuto,"Carenza di serotonina???: Gentili medici, sono...","Gentile utente, i disturbi da lei riferiti, an...",Psicoterapia,MediciItalia,Salute Mentale
2,3,https://www.medicitalia.it/consulti/psicoterap...,2007-02-12,2007-02-13,Autore sconosciuto,Autore sconosciuto,"Ora, da circa 2 settimane mi capita di sveglia...","Gentile utente, anche in considerazione della ...",Psicoterapia,MediciItalia,Salute Mentale
3,4,https://www.medicitalia.it/consulti/psicoterap...,2008-02-04,2008-02-04,Autore sconosciuto,Dr. Francesco Saverio Ruggiero Referente Scien...,Dal 2004 sono in cura da un neurologo per atta...,"Gentile utente, il trattamento psicoterapeutic...",Psicoterapia,MediciItalia,Salute Mentale
4,5,https://www.medicitalia.it/consulti/psicoterap...,2007-06-29,2007-06-29,Autore sconosciuto,Dr.ssa Roberta Cacioppo,"Non ho più voglia di vivere, voglio morire: Ho...","Gentile utente, perchè parla di lottare contro...",Psicoterapia,MediciItalia,Salute Mentale


In [ ]:
reformulated_df.head(5)

,ID,URL,data_domanda,testo_domanda,testo_risposta,testo_risposta_riformulata,categoria,categoria_generale
0,1,https://www.medicitalia.it/consulti/psicoterap...,2007-12-16,Psichiatri: per fare psicoterapia necessitano ...,"Gentile utente, uno psichiatra dopo la special...",Uno psichiatra dopo la specializzazione può is...,Psicoterapia,Salute Mentale
1,2,https://www.medicitalia.it/consulti/psicoterap...,2007-02-08,"Carenza di serotonina???: Gentili medici, sono...","Gentile utente, i disturbi da lei riferiti, an...",Il disturbo da panico può essere una causa pro...,Psicoterapia,Salute Mentale
2,3,https://www.medicitalia.it/consulti/psicoterap...,2007-02-12,"Ora, da circa 2 settimane mi capita di sveglia...","Gentile utente, anche in considerazione della ...",La sintomatologia che descrive può essere rico...,Psicoterapia,Salute Mentale
3,4,https://www.medicitalia.it/consulti/psicoterap...,2008-02-04,Dal 2004 sono in cura da un neurologo per atta...,"Gentile utente, il trattamento psicoterapeutic...",Il trattamento psicoterapeutico per il disturb...,Psicoterapia,Salute Mentale
4,5,https://www.medicitalia.it/consulti/psicoterap...,2007-06-29,"Non ho più voglia di vivere, voglio morire: Ho...","Gentile utente, perchè parla di lottare contro...",Se si considera di avere problemi che non poss...,Psicoterapia,Salute Mentale


## Extract a sample of 30 QA pairs from each categories

In [ ]:
# Select 30 lines for each category
selected_rows = original_df.groupby("categoria").apply(lambda x: x.head(30)).reset_index(drop=True)

# Extract selected IDs
selected_ids = selected_rows["ID"].unique()

# Filter the two dataframes using only selected IDs
# filtered_original = original_df[original_df["ID"].isin(selected_ids)].reset_index(drop=True)
filtered_reformulated = reformulated_df[reformulated_df["ID"].isin(selected_ids)].reset_index(drop=True)

<ipython-input-8-85aae7dc5e3a>:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  selected_rows = original_df.groupby("categoria").apply(lambda x: x.head(30)).reset_index(drop=True)


In [ ]:
filtered_reformulated.head(5)

,ID,URL,data_domanda,testo_domanda,testo_risposta,testo_risposta_riformulata,categoria,categoria_generale
0,1,https://www.medicitalia.it/consulti/psicoterap...,2007-12-16,Psichiatri: per fare psicoterapia necessitano ...,"Gentile utente, uno psichiatra dopo la special...",Uno psichiatra dopo la specializzazione può is...,Psicoterapia,Salute Mentale
1,2,https://www.medicitalia.it/consulti/psicoterap...,2007-02-08,"Carenza di serotonina???: Gentili medici, sono...","Gentile utente, i disturbi da lei riferiti, an...",Il disturbo da panico può essere una causa pro...,Psicoterapia,Salute Mentale
2,3,https://www.medicitalia.it/consulti/psicoterap...,2007-02-12,"Ora, da circa 2 settimane mi capita di sveglia...","Gentile utente, anche in considerazione della ...",La sintomatologia che descrive può essere rico...,Psicoterapia,Salute Mentale
3,4,https://www.medicitalia.it/consulti/psicoterap...,2008-02-04,Dal 2004 sono in cura da un neurologo per atta...,"Gentile utente, il trattamento psicoterapeutic...",Il trattamento psicoterapeutico per il disturb...,Psicoterapia,Salute Mentale
4,5,https://www.medicitalia.it/consulti/psicoterap...,2007-06-29,"Non ho più voglia di vivere, voglio morire: Ho...","Gentile utente, perchè parla di lottare contro...",Se si considera di avere problemi che non poss...,Psicoterapia,Salute Mentale


In [ ]:
filtered_reformulated['testo_domanda'][1]

'Carenza di serotonina???: Gentili medici, sono una ragazza universitaria di 23 anni, assillata da circa 7 anni a questa parte da un problema che nessuno riesce a risolvere! Mi spiego meglio: ogni anno, dal mese di ottobre/novembre al mese di dicembre, soffro di strani disturbi che molti specialisti hanno associato a "carenza di serotonina" con attacchi di panico e ansia, altri alla conseguente mancanza del padre che è sparito quando io avevo poco più di 3 anni, altri ancora a disturbi neuro-vegetativi...! I miei disturbi sono:  mancanza di sensibilità in tutto il corpo, (NB: la mia pelle non si brucia con l\'acqua bollente, perchè non la percepisco!) pesantezza alla testa che non mi abbandona mai durante quel periodo, spossatezza, pressione bassa, ansia e attacchi di panico, pianto, mancanza di concentrazione e perdita di memoria, perdita di peso, svenimenti, fascicolazioni muscolari, tremori e insonnia. Ho fatto molti esami, accertamenti di vario tipo e perfino risonanza magnetica pe

## ITALIAN-NER-XXL

In [ ]:
from google.colab import userdata
userdata.get('HF_TOKEN')

In [ ]:
# CONFIG
device = 0 if torch.cuda.is_available() else -1
print(f"💻 Device in uso: {'GPU' if device == 0 else 'CPU'}")


# NER MODEL
tokenizer = AutoTokenizer.from_pretrained("DeepMount00/Italian_NER_XXL")
model = AutoModelForTokenClassification.from_pretrained("DeepMount00/Italian_NER_XXL", ignore_mismatched_sizes=True)
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple", device=device)


# TEXT GENERATION MODEL
text_gen_pipe = pipeline(
    "text-generation",
    model="google/gemma-2-9b-it",
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": {"load_in_4bit": True, "bnb_4bit_compute_dtype": torch.float16}
    },
)


# PII LABELS
PII_LABELS = {
    "NOME", "COGNOME", "CODICE_FISCALE", "EMAIL", "NUMERO_TELEFONO", "INDIRIZZO", "LUOGO",
    "DATA_NASCITA", "NUMERO_DOCUMENTO", "P_IVA", "NUMERO_CARTA", "CVV", "IBAN", "BIC",
    "NUMERO_CONTO", "BANCA", "URL", "MAC", "IMEI", "IP", "IPV6_1", "PASSWORD", "STATO_CIVILE"
}

💻 Device in uso: GPU


tokenizer_config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/235k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/724k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

Device set to use cuda:0


### Prompt Tuning

In [ ]:
test_prompt = """ Rendi anonimo il seguente testo, solo se strettamente necessario.
                  Testo:
                  "Contatta il Centro di Salute Mentale di Angri per una consulenza con un professionista qualificato."

                  IMPORTANTE: L'anonimizzazione deve essere eseguita solo se la parola segnalata rappresenta effettivamente un'entità sensibile nel contesto medico.
                  Se la classificazione fornita dal modello NER è errata o la parola non è sensibile in quel contesto, lascia il testo invariato.

                  Parole da considerare per l'anonimizzazione (fornite da un modello NER): "Angri"
                  Tipologia di entità rilevata: "LUOGO"

                  NOTA: Le entità indicate dal modello NER potrebbero essere classificate in modo errato.
                  In tal caso, non alterare il testo e restituisci la versione originale senza modifiche.
                  L’obiettivo è mantenere la semantica del testo invariata e applicare l'anonimizzazione solo quando è realmente necessaria.

                  Testo risultante:
              """

output = text_gen_pipe(test_prompt, max_new_tokens=100)[0]["generated_text"]
cleaned = output.split("Testo risultante:")[-1].strip().replace('"', '')
cleaned

'Contatta il Centro di Salute Mentale per una consulenza con un professionista qualificato.'

### Complete Anonymization

In [ ]:
import re


# PII EXTRACTION with position
def extract_pii_with_offsets(text):
    if not isinstance(text, str) or len(text.strip()) == 0:
        return []
    try:
        ents = ner_pipeline(text)
        return [
            {"word": e['word'], "start": e['start'], "end": e['end'], "label": e['entity_group']}
            for e in ents if e['entity_group'] in PII_LABELS
        ]
    except Exception as e:
        print(f"⚠️ Error on text: {text[:100]} - {e}")
        return []


# PROMPT for phrase anonymization
def build_phrase_prompt(phrase, words, labels):
    return f""" Rendi anonimo il seguente testo medico, solo se strettamente necessario.
                  Testo:
                  \"{phrase}\"

                  IMPORTANTE: L'anonimizzazione deve essere eseguita solo se la parola segnalata rappresenta effettivamente un'entità sensibile nel contesto medico.
                  Se la classificazione fornita dal modello NER è errata o la parola non è sensibile in quel contesto, lascia il testo invariato.

                  Parole da considerare per l'anonimizzazione (fornite da un modello NER): \"{words}\"
                  Tipologia di entità rilevata: \"{labels}\"

                  NOTA: Le entità indicate dal modello NER potrebbero essere classificate in modo errato.
                  In tal caso, non alterare il testo e restituisci la versione originale senza modifiche.
                  L’obiettivo è mantenere la semantica del testo invariata e applicare l'anonimizzazione solo quando è realmente necessaria.

                  Testo anonimizzato:
              """


# ANONYMIZATION of entire phrase
def anonymize_phrase(phrase, words, labels):
    prompt = build_phrase_prompt(phrase, words, labels)
    print(f"\n🔎 Prompt for words: {words} (Labels: {labels})\n{prompt}\n")
    try:
        output = text_gen_pipe(prompt, max_new_tokens=1000)[0]["generated_text"]
        print(f"💬 Raw output: {output}\n")
        cleaned = output.split("Testo anonimizzato:")[-1].strip().replace('"', '')
        print(f"✅ Anonymized phrase: '{cleaned}'\n")
        return cleaned if cleaned else "[ANONIMO]"
    except Exception as e:
        print(f"❌ Error rephrasing {words}:", e)
        return "[ANONIMO]"


# WORD REPLACEMENT in text (whole word only)
def replace_word(text, old, new):
    pattern = r'\b{}\b'.format(re.escape(old))
    return re.sub(pattern, new, text)


# MAIN FUNCTION
results = []
print("\n🚀 Starting targeted phrase-level anonymization process...")

for idx in tqdm(range(len(filtered_reformulated)), desc="Targeted anonymization..."):

    original_text = str(filtered_reformulated.loc[idx, "testo_risposta"])
    reformulated_text = str(filtered_reformulated.loc[idx, "testo_risposta_riformulata"])
    category = filtered_reformulated.loc[idx, "categoria"]

    print(f"\n\n📄 [{idx}] Category: {category}")
    print(f"🟡 Initial reformulated text:\n{reformulated_text}")

    pii_original = extract_pii_with_offsets(original_text)
    pii_reformulated = extract_pii_with_offsets(reformulated_text)

    anonymize_text = reformulated_text
    # substitutions = {}

    if not pii_reformulated:
        print("✅ No PII found.")
    else:
        print("🔍 PII found in the sentence:")

        words = []
        labels = []
        for ent in pii_reformulated:
            word = ent['word']
            label = ent['label']
            print(f"   • {word} ({label})")

            words.append(word)
            labels.append(label)

        anonymize_text = anonymize_phrase(anonymize_text, words, labels)
        # substitutions[word] = anon_word
        # updated_text = replace_word(updated_text, word, anon_word)

    pii_anonymized = extract_pii_with_offsets(anonymize_text)

    print(f"🟢 Final anonymized text:\n{anonymize_text}")
    print(f"🧹 Residual PII after anonymization: {pii_anonymized}")

    delta = (len(pii_original) - len(pii_anonymized)) / len(pii_original) if len(pii_original) > 0 else 0

    results.append({
        "ID": filtered_reformulated.loc[idx, "ID"],
        "Category": category,
        "Original_text": original_text,
        "Reformulated_text": reformulated_text,
        "Anonymized_text": anonymize_text,
        # "Substitutions": substitutions,
        "Initial_PII": [e['word'] for e in pii_original],
        "Initial_PII_count": len(pii_original),
        "Reformulated_PII": [e['word'] for e in pii_reformulated],
        "Reformulated_PII_count": len(pii_reformulated),
        "Anonymized_PII": [e['word'] for e in pii_anonymized],
        "Anonymized_PII_count": len(pii_anonymized),
        "PII_removed": len(pii_original) - len(pii_anonymized),
        "Delta_PII": round(delta, 3)
    })

# FINAL OUTPUT
refined_df = pd.DataFrame(results)
print("\n✅ DONE! Processed phrases:", len(refined_df))



🚀 Starting targeted phrase-level anonymization process...


Targeted anonymization...:   0%|          | 0/2163 [00:00<?, ?it/s]



📄 [0] Category: Psicoterapia
🟡 Initial reformulated text:
Uno psichiatra dopo la specializzazione può iscriversi all'albo degli psicoterapeuti senza alcuna altra specializzazione. In genere, gli psichiatri si orientano verso la psicoterapia se ne hanno preferenza ed a volte frequentano ulteriori scuole di specializzazione. Lo psicologo, invece, è obbligato a frequentare un corso di specializzazione per poter essere psicoterapeuta.


Targeted anonymization...:   0%|          | 2/2163 [00:00<14:37,  2.46it/s]

✅ No PII found.
🟢 Final anonymized text:
Uno psichiatra dopo la specializzazione può iscriversi all'albo degli psicoterapeuti senza alcuna altra specializzazione. In genere, gli psichiatri si orientano verso la psicoterapia se ne hanno preferenza ed a volte frequentano ulteriori scuole di specializzazione. Lo psicologo, invece, è obbligato a frequentare un corso di specializzazione per poter essere psicoterapeuta.
🧹 Residual PII after anonymization: []


📄 [1] Category: Psicoterapia
🟡 Initial reformulated text:
Il disturbo da panico può essere una causa probabile dei sintomi descritti, anche se gli esami strumentali sono stati negativi. I sintomi del disturbo da panico possono variare notevolmente nella loro espressività e combinazione. Il quadro clinico può essere causato da una combinazione di sintomi come tachicardia, difficoltà respiratorie, sensazione di nodo alla gola, sudorazioni improvvisi, sensazioni di caldo o freddo, vertigini, sensazioni di svenimento, paura di morire, impa

...

🟢 Final anonymized text:
Questi esami vengono richiesti per valutare la presenza di sintomi che potrebbero indicare una malattia autoimmune. Se si sospetta una sensibilità al glutine, è importante escludere la presenza di malattie autoimmuni o di anticorpi anti-nucleo. Tuttavia, gli anticorpi anti-nucleo positivi possono anche essere presenti in persone con solo infiammazioni o alterazioni della flora intestinale. È fondamentale considerare i livelli di infiammazione e le cause potenziali, come l'alimentazione o una recente infettione. Gli anticorpi anti-nucleo positivi possono essere un segnale importante, ma possono essere gestiti con interventi non invasivi, come modifiche al comportamento alimentare e allo stile di vita, che possono portare a un riequilibrio rapido e a una maggiore consapevolezza del proprio corpo.
🧹 Residual PII after anonymization: []


📄 [2152] Category: Tumori
🟡 Initial reformulated text:
Se hai dubbi sul seno, è sempre meglio rivolgerti a un medico specializza

Targeted anonymization...: 100%|█████████▉| 2157/2163 [34:36<00:00, 16.43it/s]

✅ No PII found.
🟢 Final anonymized text:
Un esame strumentale specifico è necessario per valutare la condizione medica. Successivamente, un'indagine urologica approfondita sarà eseguita per determinare il corretto piano di trattamento.
🧹 Residual PII after anonymization: []


📄 [2156] Category: Tumori
🟡 Initial reformulated text:
Sospetta esofagite riacutizzata. Se i sintomi non migliorano entro 8-10 giorni, è necessario rifare la gastroscopia.
✅ No PII found.
🟢 Final anonymized text:
Sospetta esofagite riacutizzata. Se i sintomi non migliorano entro 8-10 giorni, è necessario rifare la gastroscopia.
🧹 Residual PII after anonymization: []


📄 [2157] Category: Tumori
🟡 Initial reformulated text:
Un controllo periodico dell'emocromo è consigliato per garantire la salute. Un intervallo di 6 mesi è un tempo ragionevole per eseguire questo controllo.
✅ No PII found.
🟢 Final anonymized text:
Un controllo periodico dell'emocromo è consigliato per garantire la salute. Un intervallo di 6 mesi è 

Targeted anonymization...: 100%|██████████| 2163/2163 [34:36<00:00,  1.04it/s]

🟢 Final anonymized text:
Il tuo caso medico sembra avere un buon prognosis, ma è importante considerare le caratteristiche prognosticamente serie come il triplo negativo, indice di proliferazione del 40% e il peso di 75 Kg per 170 cm. È fondamentale mantenere un buon rapporto con il medico e seguire le raccomandazioni per ridurre il rischio di recidiva. Ciò include una buona alimentazione con alimenti freschi, soprattutto vegetali, e una buona attività fisica. È anche importante essere cauti con le terapie non approvate e concentrarsi sulle raccomandazioni scientifiche.
🧹 Residual PII after anonymization: []


📄 [2160] Category: Tumori
🟡 Initial reformulated text:
Il valore normale per una misura medica può variare a seconda dell'individuo, ma in generale è influenzato da valori alti e bassi. Se sei leggermente più alto della media, non è necessariamente una causa di preoccupazione, poiché il valore normale è una media di anche valori alti e bassi.
✅ No PII found.
🟢 Final anonymized te

In [ ]:
refined_df.head(5)

,ID,Category,Original_text,Reformulated_text,Anonymized_text,Initial_PII,Initial_PII_count,Reformulated_PII,Reformulated_PII_count,Anonymized_PII,Anonymized_PII_count,PII_removed,Delta_PII
0,1,Psicoterapia,"Gentile utente, uno psichiatra dopo la special...",Uno psichiatra dopo la specializzazione può is...,Uno psichiatra dopo la specializzazione può is...,[],0,[],0,[],0,0,0.0
1,2,Psicoterapia,"Gentile utente, i disturbi da lei riferiti, an...",Il disturbo da panico può essere una causa pro...,Il disturbo da panico può essere una causa pro...,[],0,[],0,[],0,0,0.0
2,3,Psicoterapia,"Gentile utente, anche in considerazione della ...",La sintomatologia che descrive può essere rico...,La sintomatologia che descrive può essere rico...,[www. psichiatria - online. it],1,[],0,[],0,1,1.0
3,4,Psicoterapia,"Gentile utente, il trattamento psicoterapeutic...",Il trattamento psicoterapeutico per il disturb...,Il trattamento psicoterapeutico per il disturb...,[],0,[],0,[],0,0,0.0
4,5,Psicoterapia,"Gentile utente, perchè parla di lottare contro...",Se si considera di avere problemi che non poss...,Se si considera di avere problemi che non poss...,"[Roberta, Cacioppo, r. cacioppo @ psicologia -...",6,[],0,[],0,6,1.0


In [ ]:
# RESULTS
print("Average PII in original responses:", refined_df['Initial_PII_count'].mean())
print("Average PII in reformulated responses:", refined_df['Reformulated_PII_count'].mean())
print("Average PII in anonymized responses:", refined_df['Anonymized_PII_count'].mean())
print("Average PII removed:", refined_df['PII_removed'].mean())
print("Average Delta_PII:", refined_df["Delta_PII"].mean())

Average PII in original responses: 0.27045769764216365
Average PII in reformulated responses: 0.07073509015256588
Average PII in anonymized responses: 0.01756819232547388
Average PII removed: 0.2528895053166898
Average Delta_PII: 0.12883356449375868


In [ ]:
# Saving the dataframe with anonymization results
refined_df.to_csv("/content/drive/MyDrive/IMD - Italian Medical Data/Anonymization Analysis.csv", index=False)
print("✅ File salvato: analisi_anonimizzazione.csv")

✅ File salvato: analisi_anonimizzazione.csv


### DEPRECATED - GLINER (UniversalNER-ITA)

In [ ]:
!pip install gliner

In [ ]:
from gliner import GLiNER


# CONFIG
device = 0 if torch.cuda.is_available() else -1
print(f"💻 Device in uso: {'GPU' if device == 0 else 'CPU'}")


# LOAD THE GLiNER MODEL
model = GLiNER.from_pretrained("DeepMount00/universal_ner_ita", device="cuda" if torch.cuda.is_available() else "cpu")


# Defining sensitive entities in clinical settings
labels = [
    "nome",
    "cognome",
    "data di nascita",
    "codice fiscale",
    "indirizzo, via e posizione",
    "numero di telefono",
    "email e sito web",
]

### Removing extra whitespaces from IMD-MCQA

In [ ]:
import json

# Carica il file JSON
with open("/content/drive/MyDrive/IMD - Italian Medical Data/Dataset/IMD-MCQA.json", "r", encoding="utf-8") as file:
    data = json.load(file)

# Funzione per pulire gli spazi in eccesso dalle risposte
def clean_answers(questions):
    for question in questions:
        for key in ["AnswerA", "AnswerB", "AnswerC", "AnswerD", "AnswerE", "Correct Answer"]:
            if key in question and isinstance(question[key], str):
                question[key] = question[key].strip()
    return questions

# Applica la funzione a tutte le categorie
for category in data:
    data[category] = clean_answers(data[category])

# Salva il file JSON modificato
with open("IMD-MCQA_cleaned.json", "w", encoding="utf-8") as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

print("Spazi in eccesso rimossi e file salvato come 'IMD-MCQA_cleaned.json'")

Spazi in eccesso rimossi e file salvato come 'IMD-MCQA_cleaned.json'
